Implement a optimization method for feature combination / selection 

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [3]:
# -- LOAD A DATASET

dataset_name = 'mr_nirp'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames
print("Number of videos: ", len(allvideo))  

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

videos = constants.get_video_settings(dataset_name)
print(videos)

Number of videos:  15
0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_940.avi
7 D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\RGB_corrected\Subject4_still_940.avi
8 D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\RG

In [7]:
# -- PARAMETER SETTING

roi = 'nose'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for mr_nirp is 60 
Testing ROI = nose with minimum 2 landmarks


# Hyperopt

https://celmore25.medium.com/automated-feature-selection-with-hyperopt-46865e1b4fce

Documentation: https://hyperopt.github.io/hyperopt/#documentation

Code: https://github.com/hyperopt/hyperopt

- It does not look like the fmin works too well when the numbe rof max iterations is less than the numbre of combinations. looks like ill have to do an exaustive search after all. 
- I can discard lips  and temple area

TODO change from run mlti methods to run dataset ? 


In [8]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [9]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run
from hyperopt.base import JOB_STATE_DONE
from hyperopt.early_stop import no_progress_loss
from itertools import chain, combinations

def get_combinations(elements, min_len=2):
    combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
    return [list(i) for i in combs]

# parameter space
parameter_space = {}
ldmks_list = rois[roi]

# ldmks_list = ['chin', 'glabella']
for ldmk in ldmks_list:
    parameter_space[ldmk] = hp.choice(ldmk, [0, 1])
print(f"Selection roi as {roi} with landmarks {ldmks_list}")
print(f"Number of possible combinations with {len(ldmks_list)} landmarks is :", 2**len(ldmks_list) - 1)

all_combinations = get_combinations(parameter_space.keys(), min_len)
for elem in all_combinations:
    elem.sort()
print(f"For min length {min_len}, number of all combinations is {len(all_combinations)}")
print(all_combinations)

valid_combinations = []
for comb in all_combinations:
    valid = True
    if 'left' in ''.join(comb):
        lefts = [s for s in comb if 'left' in s]
        for elem in lefts:
            if '_'.join(['right']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if 'right' in ''.join(comb):
        rights = [s for s in comb if 'right' in s]
        for elem in rights:
            if '_'.join(['left']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if valid:
        valid_combinations.append(comb)
print(f"For min length {min_len}, number of valid combinations is {len(valid_combinations)}")
print(valid_combinations)

Selection roi as nose with landmarks ['upper_nasal_dorsum', 'lower_nasal_dorsum', 'left_mid_nasal_sidewall', 'right_mid_nasal_sidewall', 'left_lower_nasal_sidewall', 'right_lower_nasal_sidewall', 'nasal_tip', 'soft_triangle', 'left_ala', 'right_ala']
Number of possible combinations with 10 landmarks is : 1023
For min length 2, number of all combinations is 1013
[['lower_nasal_dorsum', 'upper_nasal_dorsum'], ['left_mid_nasal_sidewall', 'upper_nasal_dorsum'], ['right_mid_nasal_sidewall', 'upper_nasal_dorsum'], ['left_lower_nasal_sidewall', 'upper_nasal_dorsum'], ['right_lower_nasal_sidewall', 'upper_nasal_dorsum'], ['nasal_tip', 'upper_nasal_dorsum'], ['soft_triangle', 'upper_nasal_dorsum'], ['left_ala', 'upper_nasal_dorsum'], ['right_ala', 'upper_nasal_dorsum'], ['left_mid_nasal_sidewall', 'lower_nasal_dorsum'], ['lower_nasal_dorsum', 'right_mid_nasal_sidewall'], ['left_lower_nasal_sidewall', 'lower_nasal_dorsum'], ['lower_nasal_dorsum', 'right_lower_nasal_sidewall'], ['lower_nasal_dors

In [6]:
from hyperopt.early_stop import no_progress_loss

class HpOptBinarySelect:
    def __init__(self, videos, dataset, winsize, patch_size, space, min_len, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.parameter_space = space
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.all_combinations = get_combinations(space.keys(), min_len)
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.load_ground_truth(videos)

    def get_combinations(elements, min_len=2):
        combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
        return [list(i) for i in combs]

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def objective(self,params):
        ldmks_list = [i for i,j in params.items() if j==1]
        print(f"LANDMARKS: {ldmks_list}")

        # TODO maybe i should just define params as such
        if ldmks_list not in valid_combinations:
            return {'loss': np.nan,'status': STATUS_FAIL}
        self.all_combinations = valid_combinations

        # Less than 2 landmarks tested
        if len(ldmks_list) < min([len(combs) for combs in self.all_combinations]):
            return {'loss': np.nan,'status': STATUS_FAIL}
    
        # Get past landmarks that were tested
        past_param_values = [trial['misc']['vals'] for trial in self.trials._dynamic_trials if trial['result']['status'] == 'ok']
        past_landmarks = []
        for value in past_param_values:
            past_landmarks.append([i for i,j in value.items() if j[0]==1])

        # All combinations have been tested
        if len(past_param_values) == len(self.all_combinations):
            print(f"All {len(past_param_values)} combinations have been tested ")
            self.trials._fmin_cancelled = True
            return {'loss': np.nan,'status': STATUS_FAIL}

        # Remove duplicate trials
        if ldmks_list in past_landmarks:
            return {'loss': np.nan,'status': STATUS_FAIL}
        
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            print(videoName)
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=ldmks_list, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        print(f"Total loss for {ldmks_list}: {sum(losses)}")
        return {'loss': sum(losses), 'status': STATUS_OK}

    def optimize(self, max_evals=20):
        self.trials = Trials()
        best = fmin(fn=self.objective,
                    space=self.parameter_space,
                    algo=tpe.suggest,
                    max_evals=max_evals,
                    trials=self.trials,
                    # early_stop_fn=no_progress_loss(10),
                    )
        return space_eval(self.parameter_space, best)
    
FMinIter.run = run
Trials.refresh = refresh

In [7]:
# np.arange(0, len(dataset.videoFilenames))
pl = custom_pipeline.LandmarksPipeline()
hpobj = HpOptBinarySelect(
    videos=videos['MOTION'], dataset=dataset, winsize=8, patch_size=patch_size, 
    space=parameter_space, min_len=2, pipeline=pl, methods=['cupy_CHROM'], verb=False
)

best = hpobj.optimize(max_evals=20)
out = [i for i,j in best.items() if j==1]
print(f'Final number of features {len(out)}: ', out)

4: Unusable data.
16: Unusable data.
Video name:  {22} ['D:/datasets_rppg/lgi_ppgi\\alex\\alex_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_talk\\cv_camera_sensor_stream_handler.avi', 'D:/data

In [8]:
df = hpobj.res.reset_index(drop=True)

# video filenames and indexes
if dataset_name == 'lgi_ppgi':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
if dataset_name == 'mr_nirp':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

# add DTW
from fastdtw import fastdtw
df.insert(11, 'DTW', None)
for row in df.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df.loc[row.Index, 'DTW'] = [distance]

df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
df['dataset'] = dataset_name
df['ROI'] = roi
df = df.drop(columns=['sigFilename', 'bpmES_mad']).reset_index(drop=True)

print(df.ROI.unique(), df.landmarks.unique().size)
print("Number of unique combinations tested: ", df.landmarks.apply(lambda x: x.unique().size))
print("Best score achieved with : ", out)
print(f"Number of NaN rows {df['bpmGT'].isna().sum()} ({df[df['bpmGT'].isna()]['videoFilename'].iloc[0] if df['bpmGT'].isna().sum().sum() > 0 else None})")
df[df['landmarks'].apply(lambda x: x == out)][['videoFilename', 'RMSE' ,'MAE', 'PCC']]

# df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
# df['PCC'] = df['PCC'].apply(lambda x: abs(x[0])) 
# df[['landmarks', 'videoIdx', 'MAE', 'RMSE', 'PCC']].groupby('landmarks').mean().sort_values(by='RMSE')

Number of unique combinations tested:  6
Best score achieved with :  ['left_nasolabial_fold', 'right_nasolabial_fold']
Number of NaN rows 0 (None)


,videoFilename,RMSE,MAE,PCC
0,alex_gym,[39.5416385965721],[31.035729667467947],[0.22878178597159798]
1,alex_resting,[3.8677199413728505],[1.6827799479166663],[0.37891532249938015]
2,alex_rotation,[14.764583975285912],[10.172807835820896],[0.4765042514829794]
3,alex_talk,[17.740009961721405],[11.803245367005815],[0.18620010136739149]
4,angelo_resting,[0.9338586121923793],[0.7683064088983055],[0.9424148739173003]
5,angelo_rotation,[7.071520330132991],[2.575734105603449],[0.09262214876540269]
6,angelo_talk,[10.208961646337979],[5.662620192307692],[0.1606461497326001]
7,cpi_gym,[4.018171139783883],[2.0510615190582966],[0.9501840628355749]
8,cpi_resting,[1.070340669701415],[0.8067608173076929],[0.9165364080140911]
9,cpi_rotation,[4.3051541873544945],[2.4580747003424657],[0.3221253301415303]


In [9]:
df.head(1)

,method,dataset,sigFilename,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,bpmES_mad,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,,,0,alex_gym,[39.5416385965721],[31.035729667467947],[0.22878178597159798],[0.1444692735540005],[-6.204492030402598],[98.38671875],[8727.3505859375],"[4.0283203125, 1.0986328125, 0.3662109375, 1.4...","[101.0, 100.0, 97.0, 94.5, 93.0, 93.0, 93.0, 9...","[53.1005859375, 52.001953125, 50.1708984375, 4...",,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",230.02995,"[left_nasolabial_fold, right_nasolabial_fold]"


In [10]:
print("saving into ", f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5')
df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')
# df_old = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
# df_new = pd.concat([df_old, df], ignore_index=True)
# df_new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')

x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name}/{dataset_name}_landmarks_in_roi.h5', key='df')
df = pd.concat([df, x], axis=0)
print(df.ROI.unique(), df.landmarks.unique().size)
df.head()
# df.to_hdf(f'../results/test_landmarks/h5/{dataset_name}/{dataset_name}_landmarks_in_roi.h5', key='df')

saving into  ../results/test_landmarks/LGI_PPGI_mustache.h5


C:\Users\erolland\AppData\Local\Temp\3\ipykernel_24612\707446877.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'DTW', 'MAD', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks'],
      dtype='object')]

  df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_rotation_{roi}.h5', key='df', mode='w')


In [48]:
x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df')
# x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
print(x.videoIdx.unique())
print(x.landmarks.unique().size)

[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23]
5


In [49]:
x.head(2)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,lgi_ppgi_forehead,0,alex_gym,[24.665734276197384],[14.951978165064103],[0.556308586949296],[0.519186543628614],[-4.341552955811867],[92.330078125],[5114.220703125],"[1.0986328125, 15.380859375, 4.7607421875, 8.0...","[101.0, 100.0, 97.0, 94.5, 93.0, 93.0, 93.0, 9...","[56.0302734375, 71.044921875, 86.7919921875, 6...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",207.187744,"(left_lower_lateral_forehead, right_lower_late..."
1,cupy_CHROM,lgi_ppgi_forehead,1,alex_resting,[1.4564609910056892],[0.948449337121212],[0.9288874300939538],[0.9252186060984902],[2.4762156521407603],[5.65234375],[45.38281250000001],"[0.732421875, 0.732421875, 0.3662109375, 0.732...","[68.5, 68.5, 68.0, 67.0, 67.0, 67.0, 67.0, 66....","[67.0166015625, 67.3828125, 67.0166015625, 66....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",37.899232,"(left_lower_lateral_forehead, right_lower_late..."


# Test Hyperopt

In [21]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run

FMinIter.run = run
Trials.refresh = refresh
trials = Trials()

def func(params):
    print(f"RUN {params}")
    past_param_values = [trial['misc']['vals'] for trial in trials._dynamic_trials if trial['result']['status'] == 'ok']
    print(f"Past landmarks: {past_param_values}")

    for x in trials.trials[:-1]:
        space_point_index = dict([(key,value[0]) for key,value in x['misc']['vals'].items() if len(value)>0])
        print(space_point_index)
        if params == space_eval(param_space,space_point_index):
            print("Same: ", params, space_eval(param_space,space_point_index))
            loss = x['result']['loss']
            return {'loss': loss, 'status': STATUS_FAIL}

    if len(past_param_values) >= 3:
        print(trials.trials[0])
        print(f"All combinations have been tested {len(past_param_values)}")
        print(f"first one should be safe {trials.trials[0]} ")
        if hasattr(trials.trials, "_fmin_cancelled"):
            print(f"Cancelling trials {trials._fmin_cancelled}")
        trials._fmin_cancelled = True
        return {"loss":np.nan,"status": STATUS_FAIL}
    
    if (params < 0):
        return {"loss": (params - 3) ** 2, "status": STATUS_OK}  
    else:
        print(f"ERROR ERROR ERROR {len(trials._dynamic_trials)}")
        return {"loss":np.nan,"status": STATUS_FAIL}

param_space = hp.randint("x", -2, 2)

argmin = fmin(
    fn=func,
    space=param_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    catch_eval_exceptions = False,
)

print(argmin)

RUN 0                                                
Past landmarks: []                                   
ERROR ERROR ERROR 1                                  
RUN -1                                               
Past landmarks: []                                   
RUN -2                                               
Past landmarks: [{'x': [-1]}]                                    
{'x': -1}                                                        
RUN -2                                                           
Past landmarks: [{'x': [-1]}, {'x': [-2]}]                       
{'x': -1}                                                        
{'x': -2}                                                        
Same:                                                            
-2                                                               
-2                                                               
RUN 0                                                            
Past landmarks: 

KeyboardInterrupt: 

In [14]:
print("Number of trials: ", len(trials._dynamic_trials), len(trials.trials))
trials.trials

Number of trials:  8 3


[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 25.0, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'x': [0]},
   'vals': {'x': [-2]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 11, 21, 13, 25, 26, 389000),
  'refresh_time': datetime.datetime(2023, 11, 21, 13, 25, 26, 389000)},
 {'state': 2,
  'tid': 5,
  'spec': None,
  'result': {'loss': 16.0, 'status': 'ok'},
  'misc': {'tid': 5,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'x': [5]},
   'vals': {'x': [-1]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 11, 21, 13, 25, 26, 409000),
  'refresh_time': datetime.datetime(2023, 11, 21, 13, 25, 26, 419000)},
 {'state': 2,
  'tid': 6,
  'spec': None,
  'result': {'loss': 25.0, 'status': 'ok'},
  'misc': {'tid': 6,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   